# Setup

In [7]:
import pandas as pd
import numpy as np
import pickle
import time
from custom_methods import preprocessing

datapath = '../Data/'

## Start Timer

In [22]:
startTime = time.time()

# Billing Data

In [3]:
# Load Billing Data
fileyears = ['2015', '2016', '2017', '2018', '2019', '2020']
path = 'SpaData_'
df = pd.read_csv(datapath+path+fileyears[0]+'_Anon.csv')
for fileyear in fileyears[1:]:
    df = df.append(pd.read_csv(datapath+path+fileyear+'_Anon.csv'))

rows = len(df)
accts = df.SPA_ACCT_ID.nunique()
print(f'Length: {rows}')
print(f'Accounts: {accts}')
df.head()

Length: 3821082
Accounts: 98054


,ARREARSMONTH,RES_EL_CUR120_DAYS,RES_EL_CUR22_DAYS,RES_EL_CUR30_DAYS,RES_EL_CUR60_DAYS,RES_EL_CUR90_DAYS,RES_EL_CUR_BAL_AMT,RES_EL_OVER_120_DAYS,RES_GAS_CUR120_DAYS,RES_GAS_CUR22_DAYS,...,SEVERANCE_ELECTRIC,SEVERANCE_GAS,MONTHID,CITY_TOT_DUE,CITY_30_DAYS_PAST_DUE_AMT,CITY_60_DAYS_PAST_DUE_AMT,CITY_90_DAYS_PAST_DUE_AMT,SPA_PREM_ID,SPA_ACCT_ID,COVID_REMINDER
0,201512,0.0,0.0,90.02,0.00,0.0,90.02,0.0,0.0,0.0,...,1.0,0.0,4.885320e+10,131.59,0.00,0.0,0.0,3.0,139.0,NaN
1,201512,0.0,0.0,72.37,0.00,0.0,72.37,0.0,0.0,0.0,...,NaN,NaN,8.903202e+09,186.60,0.00,0.0,0.0,33.0,181.0,NaN
2,201512,0.0,0.0,528.84,81.34,0.0,610.18,0.0,0.0,0.0,...,1.0,0.0,9.672015e+08,331.86,130.83,72.7,0.0,37.0,17.0,NaN
3,201512,0.0,54.0,0.00,0.00,0.0,54.00,0.0,0.0,80.0,...,NaN,NaN,4.180520e+10,105.81,0.00,0.0,0.0,73.0,173.0,NaN
4,201512,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,...,NaN,NaN,5.075220e+10,98.11,0.00,0.0,0.0,110.0,140.0,NaN


In [4]:
df.columns.to_list()

['ARREARSMONTH',
 'RES_EL_CUR120_DAYS',
 'RES_EL_CUR22_DAYS',
 'RES_EL_CUR30_DAYS',
 'RES_EL_CUR60_DAYS',
 'RES_EL_CUR90_DAYS',
 'RES_EL_CUR_BAL_AMT',
 'RES_EL_OVER_120_DAYS',
 'RES_GAS_CUR120_DAYS',
 'RES_GAS_CUR22_DAYS',
 'RES_GAS_CUR30_DAYS',
 'RES_GAS_CUR60_DAYS',
 'RES_GAS_CUR90_DAYS',
 'RES_GAS_CUR_BAL_AMT',
 'RES_GAS_OVER_120_DAYS',
 'BREAK_ARRANGEMENT',
 'BREAK_PAY_PLAN',
 'CALL_OUT',
 'CALL_OUT_MANUAL',
 'DUE_DATE',
 'FINAL_NOTICE',
 'PAST_DUE',
 'SEVERANCE_ELECTRIC',
 'SEVERANCE_GAS',
 'MONTHID',
 'CITY_TOT_DUE',
 'CITY_30_DAYS_PAST_DUE_AMT',
 'CITY_60_DAYS_PAST_DUE_AMT',
 'CITY_90_DAYS_PAST_DUE_AMT',
 'SPA_PREM_ID',
 'SPA_ACCT_ID',
 'COVID_REMINDER']

## Rename Attributes

In [5]:
df = df.rename({'ARREARSMONTH':'MONTH'}, axis=1)
df = df.drop('MONTHID', axis=1)

## Reformat Dates

In [6]:
print(f'Earliest Month: {df.MONTH.min()}')
print(f'Latest Month: {df.MONTH.max()}')

Earliest Month: 201512
Latest Month: 202012


Use December, 2015 as month 0 - this is the earliest month in the billing data

In [10]:
df.MONTH = df.MONTH.apply(lambda x: preprocessing.date_map(date=x, relative_to=201512, format='yyyymm'))
print(f'Earliest Month: {df.MONTH.min()}')
print(f'Latest Month: {df.MONTH.max()}')

Earliest Month: 0
Latest Month: 60


# Service Agreements Data

In [64]:
sa = pd.read_csv(datapath+'ServiceAgreements_Anon.csv').\
    rename({'spa_prem_id':'SPA_PREM_ID', 'spa_acct_id':'SPA_ACCT_ID', 'spa_per_id':'SPA_PER_ID', 'homelessMatch':'CMIS_MATCH', 'EnrollDate':'ENROLL_DATE', 'apartment':'APARTMENT'}, axis=1)
rows = len(sa)
ppl = sa.SPA_PER_ID.nunique()
accts = sa.SPA_ACCT_ID.nunique()
pos_ppl = sa[sa.CMIS_MATCH == True].SPA_PER_ID.nunique()

print(f'Rows: {rows}')
print(f'People: {ppl}')
print(f'Accounts: {accts}')
print(f'Positive Cases: {pos_ppl}')
sa.head()

Rows: 709068
People: 305480
Accounts: 270990
Positive Cases: 2386


,SPA_PREM_ID,SPA_ACCT_ID,spa_sa_id,SPA_PER_ID,ACCT_REL_TYPE_CD,CMIS_MATCH,START_DT,END_DT,SA_TYPE_DESCR,Class,APARTMENT,ENROLL_DATE
0,115011.0,197077.0,394613.0,81568.0,MAIN,NaN,2018-07-28,NaN,Residential Electric WA,RESIDENTIAL,False,NaN
1,115011.0,197077.0,394613.0,226934.0,COTENANT,NaN,2018-07-28,NaN,Residential Electric WA,RESIDENTIAL,False,NaN
2,144240.0,103592.0,207601.0,39347.0,COTENANT,NaN,2018-07-25,2019-05-31,Residential Electric WA,RESIDENTIAL,False,NaN
3,144240.0,103592.0,207601.0,57810.0,MAIN,NaN,2018-07-25,2019-05-31,Residential Electric WA,RESIDENTIAL,False,NaN
4,83426.0,74182.0,148569.0,272447.0,MAIN,NaN,2018-07-22,2019-12-02,Residential Electric WA,RESIDENTIAL,True,NaN


## Drop Unwanted Attributes, Reformat

In [65]:
sa = sa.drop(['spa_sa_id', 'START_DT', 'END_DT', 'SA_TYPE_DESCR', 'Class', 'APARTMENT'], axis=1)
# Apartment attribute unreliable
sa.CMIS_MATCH = sa.CMIS_MATCH.replace(to_replace=np.nan, value=False).astype('bool')
sa.ENROLL_DATE = sa.ENROLL_DATE.apply(lambda x: preprocessing.date_map(date=x, relative_to='2015-12-01', format='yyyy-mm-dd'))
sa = sa.drop_duplicates()

## Multiple Enroll Dates
Choose first - most interested in predicting first experience of homelessness

In [52]:
sa.groupby(['SPA_PER_ID', 'SPA_PREM_ID', 'SPA_ACCT_ID']).size().value_counts()

1    441772
2        91
dtype: int64

In [53]:
sa.groupby(['SPA_PER_ID', 'SPA_PREM_ID', 'SPA_ACCT_ID', 'ENROLL_DATE']).size().value_counts()

1    3031
dtype: int64

In [66]:
min_enroll = sa.groupby(['SPA_PER_ID', 'SPA_PREM_ID', 'SPA_ACCT_ID']).ENROLL_DATE.min()
sa = sa.set_index(['SPA_PER_ID', 'SPA_PREM_ID', 'SPA_ACCT_ID'])
sa.ENROLL_DATE.update(min_enroll)
sa = sa.drop_duplicates()
sa.groupby(['SPA_PER_ID', 'SPA_PREM_ID', 'SPA_ACCT_ID']).size().value_counts()

1    255
2      2
dtype: int64

In [67]:
sa.ENROLL_DATE.head()

SPA_PER_ID  SPA_PREM_ID  SPA_ACCT_ID
81568.0     115011.0     197077.0      NaN
226934.0    115011.0     197077.0      NaN
268593.0    142036.0     34033.0       NaN
32373.0     25168.0      248139.0      NaN
17409.0     76264.0      187659.0      NaN
Name: ENROLL_DATE, dtype: float64

In [58]:
first_2 = sa.iloc[:2]
first_2.ENROLL_DATE = [1.0, 2.0]
first_2

C:\Users\Colin\anaconda3\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,,,ACCT_REL_TYPE_CD,CMIS_MATCH,ENROLL_DATE
SPA_PER_ID,SPA_PREM_ID,SPA_ACCT_ID,,,
81568.0,115011.0,197077.0,MAIN,False,1.0
226934.0,115011.0,197077.0,COTENANT,False,2.0


In [59]:
sa.update(first_2)
sa.head()

,,,ACCT_REL_TYPE_CD,CMIS_MATCH,ENROLL_DATE
SPA_PER_ID,SPA_PREM_ID,SPA_ACCT_ID,,,
81568.0,115011.0,197077.0,MAIN,False,1.0
226934.0,115011.0,197077.0,COTENANT,False,2.0
39347.0,144240.0,103592.0,COTENANT,False,NaN
57810.0,144240.0,103592.0,MAIN,False,NaN
272447.0,83426.0,74182.0,MAIN,False,NaN


# Geo Data - Avista

In [68]:
geo = pd.read_csv(datapath+'GeoData_Anon.csv').rename({'spa_prem_id':'SPA_PREM_ID'}, axis=1)

geo = geo.drop(["TRACT_GEOID", "BLOCKGROUP_GEOID_Data"], axis=1).drop_duplicates()

# NOTE: BLOCKGROUP_GEOID and BLOCKGROUP_GEOID_Data contain the same blockgroup number
print(f'Total Records: {len(geo)}')
print(f'Total Premises: {geo.SPA_PREM_ID.nunique()}')
print(f"Contains NaN's: {geo.isnull().any().any()}")
geo.head()

Total Records: 155538
Total Premises: 155538
Contains NaN's: False


,SPA_PREM_ID,BLOCKGROUP_GEOID,POSTAL
0,24381.0,530630112013,99208
1,71746.0,530630024001,99201
2,148291.0,530630024001,99201
3,142249.0,530630105032,99208
4,33506.0,530630106024,99208


In [72]:
df = df.join(geo.set_index('SPA_PREM_ID'), on=['SPA_PREM_ID'], how='left')
del geo

# Multi-Family Dwellings

In [74]:
dwellings = pd.read_csv(datapath+'MultiFamilyDwellingIDs_Anon.csv').rename({'spa_prem_id':'SPA_PREM_ID', 'multi_dwell_id':'MULTI_DWELL_ID'}, axis=1)
dwellings.head()

,SPA_PREM_ID,MULTI_DWELL_ID
0,92052,1
1,52062,1
2,74324,1
3,27282,1
4,21103,1


In [ ]:
df = df.join(dwellings.set_index('SPA_PREM_ID'), on=['SPA_PREM_ID'], how='left')
del dwellings

# Geo Data - Census
Using data from 2015

In [76]:
sub_datapath = datapath+'CensusData/'
match_col = 'BLOCKGROUP_GEOID'

## Aggregate Income
US Census Table: B19025

In [78]:
agg_income = pd.read_csv(sub_datapath+'AggIncome/ACSDT5Y2015.B19025_data_with_overlays_2021-04-18T191340.csv')
print(agg_income.isnull().sum())
agg_income.head()

GEO_ID         0
NAME           0
B19025_001E    1
B19025_001M    1
dtype: int64


,GEO_ID,NAME,B19025_001E,B19025_001M
0,id,Geographic Area Name,Estimate!!Aggregate household income in the pa...,Margin of Error!!Aggregate household income in...
1,1500000US530630002001,"Block Group 1, Census Tract 2, Spokane County,...",11310900,3953586
2,1500000US530630002002,"Block Group 2, Census Tract 2, Spokane County,...",7976900,3043131
3,1500000US530630002003,"Block Group 3, Census Tract 2, Spokane County,...",8076300,4173907
4,1500000US530630002004,"Block Group 4, Census Tract 2, Spokane County,...",23570500,7029237


In [79]:
agg_income.drop(0, axis=0, inplace=True)
newcol = "AGG_INCOME_GEO"
agg_income.rename({"B19025_001E":newcol}, axis=1, inplace=True)

agg_income[match_col] = agg_income["GEO_ID"].map(preprocessing.geoid_map).astype('int64')
agg_income.set_index(match_col, inplace=True)

df = df.join(agg_income[newcol], how='left', on=match_col)

del agg_income

## Earnings
US Census Table: B19051

In [80]:
earnings = pd.read_csv(sub_datapath+'Earnings/ACSDT5Y2015.B19051_data_with_overlays_2021-04-12T234426.csv')
print(earnings.isnull().sum())
earnings.head()

GEO_ID         0
NAME           0
B19051_001E    0
B19051_001M    0
B19051_002E    0
B19051_002M    0
B19051_003E    0
B19051_003M    0
dtype: int64


,GEO_ID,NAME,B19051_001E,B19051_001M,B19051_002E,B19051_002M,B19051_003E,B19051_003M
0,id,Geographic Area Name,Estimate!!Total,Margin of Error!!Total,Estimate!!Total!!With earnings,Margin of Error!!Total!!With earnings,Estimate!!Total!!No earnings,Margin of Error!!Total!!No earnings
1,1500000US530630002001,"Block Group 1, Census Tract 2, Spokane County,...",330,73,255,76,75,34
2,1500000US530630002002,"Block Group 2, Census Tract 2, Spokane County,...",325,94,110,60,215,74
3,1500000US530630002003,"Block Group 3, Census Tract 2, Spokane County,...",270,87,177,83,93,43
4,1500000US530630002004,"Block Group 4, Census Tract 2, Spokane County,...",417,90,351,90,66,34


In [83]:
earnings = earnings.drop(0, axis=0)
newcol = "NO_EARNINGS_GEO"
earnings[newcol] = earnings["B19051_003E"].astype('float') / earnings["B19051_001E"].astype('float')
earnings[match_col] = earnings["GEO_ID"].map(preprocessing.geoid_map).astype("int64")
earnings = earnings.set_index(match_col)

df = df.join(earnings[newcol], how='left', on=match_col)
del earnings

## Poverty
US Census Table B17021

In [84]:
poverty = pd.read_csv(sub_datapath+'Poverty/ACSDT5Y2015.B17021_data_with_overlays_2021-04-12T234708.csv')
print(poverty.isnull().any().any())
poverty.head()

False


,GEO_ID,NAME,B17021_001E,B17021_001M,B17021_002E,B17021_002M,B17021_003E,B17021_003M,B17021_004E,B17021_004M,...,B17021_031E,B17021_031M,B17021_032E,B17021_032M,B17021_033E,B17021_033M,B17021_034E,B17021_034M,B17021_035E,B17021_035M
0,id,Geographic Area Name,Estimate!!Total,Margin of Error!!Total,Estimate!!Total!!Income in the past 12 months ...,Margin of Error!!Total!!Income in the past 12 ...,Estimate!!Total!!Income in the past 12 months ...,Margin of Error!!Total!!Income in the past 12 ...,Estimate!!Total!!Income in the past 12 months ...,Margin of Error!!Total!!Income in the past 12 ...,...,Estimate!!Total!!Income in the past 12 months ...,Margin of Error!!Total!!Income in the past 12 ...,Estimate!!Total!!Income in the past 12 months ...,Margin of Error!!Total!!Income in the past 12 ...,Estimate!!Total!!Income in the past 12 months ...,Margin of Error!!Total!!Income in the past 12 ...,Estimate!!Total!!Income in the past 12 months ...,Margin of Error!!Total!!Income in the past 12 ...,Estimate!!Total!!Income in the past 12 months ...,Margin of Error!!Total!!Income in the past 12 ...
1,1500000US530630002001,"Block Group 1, Census Tract 2, Spokane County,...",928,257,475,229,429,230,140,205,...,72,50,72,50,72,50,0,12,0,12
2,1500000US530630002002,"Block Group 2, Census Tract 2, Spokane County,...",566,211,290,160,194,143,65,76,...,137,68,116,57,116,57,0,12,21,33
3,1500000US530630002003,"Block Group 3, Census Tract 2, Spokane County,...",582,186,197,96,74,49,19,31,...,164,151,89,80,23,38,66,70,75,77
4,1500000US530630002004,"Block Group 4, Census Tract 2, Spokane County,...",1035,325,198,142,161,142,0,12,...,205,137,143,73,111,69,32,41,62,90


# Get Processing Stats and Save

## Save Pickle

In [35]:
'''
filename = 'processed.pickle'
outfile = open(datapath+filename, 'wb')
pickle.dump(df, outfile)
outfile.close()
'''

## Check Numbers Retained

In [36]:
retained_rows = len(df)
retained_accts = df.SPA_ACCT_ID.nunique()

print(f'Retained {retained_rows} = {100*retained_rows/rows}% of rows.')
print(f'Retained {retained_accts} = {100*retained_accts/accts}% of P Cases.')

Retained 3810352 = 99.71918948611938% of rows.
Retained 98054 = 100.0% of P Cases.


## Total Time

In [37]:
print(f'Total Time in Seconds: {time.time()-startTime}')

Total Time in Seconds: 49.36093735694885
